# Gene regulatory network

This example show how to run the function of gene regulatory network on Stereopy. We use PySCENIC for gene regulatory network analysis. PySCENIC is a tool for inferring gene regulatory networks from single-cell transcriptomic data, and it requires the following input files: 

1. Single-cell transcriptomic data: The required input is a matrix of gene expression levels for each single cell. Each row represents a gene, and each column represents a single-cell transcriptome sample. The values in the matrix are typically gene expression levels, either in raw counts or normalized expression values.

2. Transcription factor gene list: This is a text file containing a list of transcription factor gene names of interest. PySCENIC will use these genes to infer the transcription factor-gene regulatory network. It can be downloaded through the link:
    [pySCENIC_TF_list](https://github.com/aertslab/pySCENIC/tree/master/resources) and
    [TF_lists](https://resources.aertslab.org/cistarget/tf_lists/).

3. Cis-regulatory annotation file for genes: This is an annotation file containing predicted enhancers and transcription factor binding sites for genes in their promoter regions. This file can typically be downloaded from public databases. It can be downloaded through the link:
    [cisTarget databases](https://resources.aertslab.org/cistarget/databases/).
    

4. Gene co-expression network file: This is a file containing a gene co-expression network, which includes a list of co-expressed neighbor genes and their correlation coefficients for each gene. It can be downloaded through the link:
    [Motif2TF annotations](https://resources.aertslab.org/cistarget/motif2tf/).



## Reading data

Firstly, download our [example data](https://pan.genomics.cn/ucdisk/s/BvIrye) and import Stereopy. We  use the spatial expression matrix of Bin200 of Stereo-Seq in the mouse olfactory brain for the use of **stereopy** tools for gene regulatory network analysis.


In [ ]:
import stereo as st
import warnings
warnings.filterwarnings('ignore')


tfs_fn = r'D:\xujunhao\work\stereopy\grn\test_mm_mgi_tfs.txt'
database_fn = r'D:\xujunhao\work\stereopy\grn\mm10_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather'
motif_anno_fn = r'D:\xujunhao\work\stereopy\grn\motifs-v10nr_clust-nr.mgi-m0.001-o0.0.tbl'

data_path = '../../data/SS200000135TL_D1/exon/SS200000135TL_D1.tissue.gef'


Load data to generate a StereoExpData object. **We recommend using raw gene expression matrices to infer gene regulation network.** If save is set to True in `data.tl.inference_regulatory_network`, the result of the gene regulation network will be output as *grn_output.loom*.

In [ ]:
data = st.io.read_gef(file_path=data_path, bin_size=50)

# simply type the varibale to get related information
data

data.tl.raw_checkpoint()

data.tl.inference_regulatory_network(database_fn, motif_anno_fn, tfs_fn, save=True)



We can view the results of the gene regulatory network in the `data.tl.result`

In [ ]:
data.tl.result['inference_regulatory_network'].keys()

In [ ]:
data.tl.result['inference_regulatory_network']['auc_matrix']

In [ ]:
data.tl.result['inference_regulatory_network']['adjacencies']

In [ ]:
data.tl.result['inference_regulatory_network']['regulons']

`data.plt.auc_heatmap` shows the activity of regulon in each cell (compared to other genes in the cell, the ratio of the expressed gene in the characteristic gene)

In [ ]:
data.plt.auc_heatmap(ign_res_key='inference_regulatory_network',width=28,height=28)

We can look at the expression of regulon in space by using `data.plt.spatial_scatter_by_regulon`

In [ ]:
data.plt.spatial_scatter_by_regulon(reg_name='Sp3', ign_res_key='inference_regulatory_network', dot_size=500)

`data.plt.grn_dotplot` shows the expression of regulon in different celltypes. First of all, we need to cluster the data first, or give the cell classification.

In [ ]:
data.tl.pca(use_highly_genes=False, n_pcs=30, res_key='pca')
data.tl.neighbors(pca_res_key='pca', n_pcs=30, res_key='neighbors')
data.tl.leiden(neighbors_res_key='neighbors', res_key='leiden')

data.plt.cluster_scatter(res_key='leiden')

In [ ]:
data.plt.grn_dotplot(data.tl.result['leiden'],ign_res_key='inference_regulatory_network')

`data.plt.auc_heatmap_by_group` will first filter out the regulons with the highest expression of each category (top_n_feature), and then perform zcore processing to draw the activity of the regulons in each cell

In [ ]:
data.plt.auc_heatmap_by_group(ign_res_key='inference_regulatory_network',celltype_res_key='leiden',top_n_feature=5)